# Learning on your own and project

## Outline

* Using **:type** and **:info** in GHCi

* Using Hoogle and Hackage
  - Hoogle
  - Hackage
  - Using **:i** together with Hoogle and Hackage

* Using the Haskell Wiki

* Learning to read existing code

* Walkthrough of creating a project

In this lesson, we will learn how you can explore types, functions and modules in Haskell on your own.

As we cover more and more, it is becoming clear that since Haskell is a functional programming language, it is all about functions and their composition.

When designing a Haskell program, we keep this in mind and cut up our code in these small chunks of functions. This allows us to reason about smaller portions of the code and their correctness. And, as a bonus, it keeps our code composable, so we can reuse it later.

So, it comes as no surprise that a lot of code is already written by others! It is our task to find useful functions and figure out how to use them.

## Using **:type** and **:info** in GHCi

 Before we start with this search for useful functions, we will explain how to use GHCi to figure out how a function works.

<div class="alert alert-block alert-info">
Remember, GHCi is the <b>interactive</b> shell for GHC (The Glasgow Haskell Compiler). It lets you play and execute code on the go! In this Jupyter notebook, each cell is actually a GHCi shell that we can use for small Haskell snippets. You can also open a GHCi shell in a terminal by executing `ghci` (you can quit by execution <code>:q</code>)
</div>

Two useful GHCi commands, which we extensively used in our lessons before, are `:type` and `:info`. Which also can be abbreviated as `:t` and `:i` in its use. These two let us inspect and gather information on types, function, instances and more!

To highlight their importance, we consider an example. Imagine that we want to open a file in Haskell, and we remember the existence of a function `openFile` from the `System.IO` module, but not its use. We can use GHCi together with `:t` and `:i` to figure out its usage!

In [ ]:
import System.IO

:t openFile
-- openFile :: FilePath -> IOMode -> IO Handle

:i FilePath
-- type FilePath = String

:i IOMode
-- data IOMode = ReadMode | WriteMode | AppendMode | ReadWriteMode

From this information, we see that the function `openFile` takes in a file path, which is a string. We also see that it takes a mode, from this context we deduce that these are for indicating the file system in what way we are opening this file.

Besides the above deconstruction of the algebraic data type `IOMode`, the `:i` command is also useful for exploring other Haskell types.

For instance, if we type `:i Int` we get additional information about this type. We can even see to which type classes the `Int` type belongs!

```haskell
:i Int
type Int :: *
data Int = GHC.Types.I# GHC.Prim.Int#
  	-- Defined in ‘GHC.Types’
instance Eq Int -- Defined in ‘GHC.Classes’
instance Ord Int -- Defined in ‘GHC.Classes’
instance Enum Int -- Defined in ‘GHC.Enum’
instance Num Int -- Defined in ‘GHC.Num’
instance Real Int -- Defined in ‘GHC.Real’
instance Show Int -- Defined in ‘GHC.Show’
instance Read Int -- Defined in ‘GHC.Read’
instance Bounded Int -- Defined in ‘GHC.Enum’
instance Integral Int -- Defined in ‘GHC.Real’
```

We see that `Int` is a member of many instances, with the `:Info` command we can also find out more about these instances. Let's look for instance at the `Num` type class.

```haskell
:i Num
type Num :: * -> Constraint
class Num a where
  (+) :: a -> a -> a
  (-) :: a -> a -> a
  (*) :: a -> a -> a
  negate :: a -> a
  abs :: a -> a
  signum :: a -> a
  fromInteger :: Integer -> a
  {-# MINIMAL (+), (*), abs, signum, fromInteger, (negate | (-)) #-}
  	-- Defined in ‘GHC.Num’
instance Num Word -- Defined in ‘GHC.Num’
instance Num Integer -- Defined in ‘GHC.Num’
instance Num Int -- Defined in ‘GHC.Num’
instance Num Float -- Defined in ‘GHC.Float’
instance Num Double -- Defined in ‘GHC.Float’
```

We get all the class functions and their type signatures. From the `{-# MINIMAL ... -#}` description, we even see which functions are minimally required to be a member of this instance. Conversely, this also means that any type that is an `Int`, is a member of the `Num` type class, so it has all of the above class functions available. And indeed we can negate the following

In [ ]:
numberOne :: Int
numberOne = 1
:t numberOne
negate numberOne

Lastly, at the end, all built in Haskell types that are members of this type class are listed with their module location.

## Using Hoogle and Hackage

As we saw above, with using GHCi, we can figure out many things about function and types we know exist. But it might be the case that we simply do not know about some open-source implementation of a piece of software we need. This is where Hoogle and Hackage come in.

### Hoogle

**Hoogle** (*hoogle.haskell.org*) is a Haskell API search engine, which allows you to search the Haskell libraries on Hackage by either function name, or by approximate type signature. 

This is an extension to `:info` and `:type` commands, but with Hoogle you get a more detailed and graphically friendly explanation of types and functions. 

Hoogle also enables you to search for modules contained in Haskell and see information about the functions that are defined in a specific module. 

Here are some example searches you can try out:

* **map** searches as text, finding map, concatMap, mapM
* **con map** searches for the text "map" and "con" finding concatMap, but not map
* **a -> a** searches by type, finding id :: a -> a
* **a** searches for the text "a"
* **:: a** searches for the type "a"
* **id :: a -> a** searches for the text "id" and the type "a -> a"

Additional to your search, you can add restriction on the modules that the function resides in. For example, if we search for

* **file -System** this excludes results from modules such as System.IO, System.FilePath.Windows and Distribution.System
* **foldr +Data.Map** finds results in the Data.Map module

### Hackage

**Hackage** (*hackage.haskell.org*) is the Haskell community's central package archive of open-source software. 

If you tried out Hoogle, you may have noticed that the page that loads and shows you information about the function, type, or module is actually hosted on *hackage.haskell.org*. 

If we for instance type in Hoogle **Map**, the first result we get is the **Data.Map** module. When we click on it, we get redirected to Hackage. On the beginning of this page, we see the following note:

```
Note: You should use **Data.Map.Strict** instead of this module if:

- You will eventually need all the values stored.
- The stored values don't represent large virtual data structures to be lazily computed.
```

When we click on **Data.Map.Strict**, a new page gets loaded, and we see that this module contains an updated description of all the functions and data typed that are part of the **Data.Map** module. 

Because in the dot notation of modules, we go one step into the tree when you import **Data.Map** everything from the child branches contained in **Data.Map** is also imported. 

Do not forget to check the notes at the top section of the module you are looking at, since they can state this module should use a qualified import. 

Another thing to mention about Hackage is that, when you are looking at a module as **Data.Map.Strict** you may see on the left side a table of contents that defines the module. 

Also in the top-left corner of the page, you can see the name of the package that contains this module. For the module **Data.Map.Strict** the package name is **containers** with current version **0.6.6**. 

If you go to the starting page of Hackage, click on Browse packages you can type in **containers** in the search bar and list of packages that contains this name will be displayed. 

Then you can further click on the package **containers**, and you will get some information about the package together with a list of all the modules it contains, of which **Data.Map** is just one.

### Looking up source code

When we used the command `:i Int` or `:i Num` beside the instance statements, this command showed a line: 
```haskell
-- Defined in ...
```

These comments tell you where you can find the definition of these instances. 

For the `Num` type class we have an instance for `Int` which is defined in `GHC.Num`.

You can go to Hoogle type in *GHC.Num*, click on the result and scroll down until you find the line in the instance's table `Num Int`.

Then click on the text `# source` that is on the right side of the definition, and you will see the source code where this instance is defined:
```haskell
instance Num Int where
    I# x + I# y = I# (x +# y)
    I# x - I# y = I# (x -# y)
    negate (I# x) = I# (negateInt# x)
    I# x * I# y = I# (x *# y)
    abs n  = if n `geInt` 0 then n else negate n

    signum n | n `ltInt` 0 = negate 1
             | n `eqInt` 0 = 0
             | otherwise   = 1

    fromInteger i = I# (integerToInt# i)
```

You can do the same for any function, type or instance definition that you find on Hackage.

## Using the Haskell Wiki

The Haskell wiki page can be found at *wiki.haskell.org*. It contains numerous useful articles and links that cover various areas of Haskell, such as:

* Learning Haskell
* Using Haskell
* Joining the community

You can also use the wiki search bar in the top-right corner to search for articles on the Haskell wiki page. 

Keep in mind that for a specif problem it is maybe good to do a Google search, but for learning general things about the Haskell ecosystem this is a good starting point.

A more up-to-date list of community pages and learning resources can be found on *haskell.org*:

* https://www.haskell.org/community/
* https://www.haskell.org/documentation/ 

Nevertheless, you could also find interesting educational content if you search on Google or YouTube.

A good example of learning resources that are not stated anywhere on the Haskell wiki page or the Haskell documentation page at current time of writing are: 
- the Stack Overflow tutorial https://devtut.github.io/haskell/ that provides a basic and advanced Haskell tutorial 

- the advance Haskell libraries and tutorial explanations from FP complete project: https://www.fpcomplete.com/haskell/learn/ 

## Learning to read existing code

Above, we used Hoogle and Hackage to inspect the **Data.Map.Strict** module, let's go now through some code that uses this module. After that, we will explain how you would try to understand such code. 

In [ ]:
import Data.Map as Map ( fromList, insert, toList, Map, delete )

data Book = Book 
              { author :: String
              , title :: String
              } deriving Show

book1 :: Book
book1 = Book { author = "Douglas Adams"
             , title = "The Hitchhiker's Guide to the Galaxy"
             }

book2 :: Book
book2 = Book { author = "J. R. R. Tolkien"
             , title = "Lord of the Rings"
             }

book3 :: Book
book3 = Book { author = "Frank Herbert"
             , title = "Dune"
             }

library :: Map.Map Int Book
library = Map.fromList [(1,book1),(3,book2)]

listBooks :: Map.Map Int Book -> IO ()
listBooks lib = do
    let books = Prelude.map snd (Map.toList lib)
    print books

addBook :: Map.Map Int Book -> Book -> Map.Map Int Book
addBook lib book = Map.insert (getFreeSpot 1) book lib
    where libKeys = Prelude.map fst $ Map.toList lib
          getFreeSpot spot = if spot `elem` libKeys
                             then getFreeSpot (spot + 1)
                             else spot

removeBook :: Map.Map Int Book -> Int -> Map.Map Int Book
removeBook lib id = Map.delete id lib

main :: IO ()
main = do
    putStr "Current books in library are:\n"
    listBooks library
    putStr "\nAdding a book to library:\n"
    let library1 = addBook library book3
    listBooks library1
    putStr "\nRemoving book with id = 3 from library:\n"
    listBooks $ removeBook library1 3

This little code snippet starts off with an import of functions in from the **Data.Map** module. Often in code you will find only the modules get imported without the declared functions, keep this in mind when looking at code!

<div class="alert alert-block alert-info">
For some IDEs and text editors, it is possible to add the *Haskell language server* extension which automatically parses the code and suggest changing the import statement of a module so that it lists all the used functions from the code: https://wiki.haskell.org/IDEs#Haskell_Language_Server 
</div>

After that, we see a `book` type is defined and three of these values are created. In the library type declaration, we see that a `Map` data type is used.

To discover how this works, we go to Hoogle, and search for "Map". The first result is the **Data.Map** module, but the second is the definition of the Map type.

If we click on the second result, we get the following definition:
```haskell
data Map k a 
A Map from keys k to values a.
```

In the code, we also see that the Map object is created with the `fromList` function. If we check on Hoogle `fromList +Data.Map` we get the following type signature:
```haskell
fromList :: Ord k => [(k, a)] -> Map k a
```
```
Build a map from a list of key/value pairs. If the list contains more than one value for the same key, the last value for the key is retained.
```
So, it takes in a list of tuples and turns it into a Map. The keys have to have an instance of Ord. In our case, we use `Int` for keys and `Book` type for values.

Next we have the function `listBooks` that takes in a Map which holds books and performs some IO actions. We see there we use the function `toList` from the Map module.

From Hoogle we get:
```haskell
toList :: Map k a -> [(k, a)] 
```
```
Convert the map to a list of key/value pairs
```

It takes in a Map object and returns a list of tuples. The opposite of the `fromList` function.

So, the `listBooks` function can take in our library and list all the books that are contained in the Map.

Then we come to the function `addBook` that takes in a Map which contains books, a book, and returns a Map which contains books.

The only function we do not know yet in this code is the `insert` function from the Map module. From Hoogle we get:
```haskell
insert :: Ord k => k -> a -> Map k a -> Map k a 
```
```
Insert a new key and value in the map. If the key is already present in the map, the associated value is replaced with the supplied value.
```

From the code we see, this function can update our library with a new Book, where it chooses the first free key that the library does not contain yet.

The last function that works with a Map is the `removeBook` function that takes in a Map of Books and an Int, and returns a Map of Books.

We see that it uses the `delete` function from the Map module. From Hoogle we get:
```haskell
delete :: Ord k => k -> Map k a -> Map k a 
```
```
Delete a key and its value from the map. When the key is not a member of the map, the original map is returned.
```

So, this function can delete a book from our library where we identify it with the id of the book, that are the keys in the Map.

In the end we have the `main` function that lists the books from our library, then adds one book and lists the library and in the end removes one book and lists the library again.

<div class="alert alert-block alert-info">
If you want to use a module you learned about in your future code, it is worth to check the main page of the module for comments. For example: for the `Data.Map` module we need a qualified import to make sure function do not clash from the `Prelude` module.
</div>

## Walkthrough of creating a project

In this section, we will show the creation of a small Haskell project. This project will not be finished in this notebook, but snippets of the code will be given to sketch the implementation. 

Besides these snippets, we will also give clarification on the design decisions taken along the way. The goal here is to give you a feeling for how to choose the correct types and function when building a project.

The project idea is a program that functions similar to an online web shop. It simulates a shopping basket, and it has a database with items that you can add to the shopping basket. Additionally, once the shopping is done, the program can reduce the price of the total shopping basket, depending on a membership program. For simplicity, we will only have three categories in this online shop, music, books and coffee.

So, how can we translate this idea to code? We might try to implement the Database structure as a `Map` from the module `Data.Map` described above.


This database `Map` contains three entries, one for each class of items. So, it's either one of the keys `"music"`, `"books"` or `"coffee"`.

```json
{
    "music": ...
    "books": ...
    "coffee": ...
}
```

The shop can have multiple kinds of music, books or coffee that they sell. So, we might choose for each of these categories in our database `Map` to have as their value a `Map` as well. Here, each object in a category will have as its key its index in the category `Map`.

<div class="alert alert-block alert-info">
Alternatively, you might consider a list of items instead of a <code>Map</code>, but note that <code>Map</code> is easier to use as a database since you can access the data via keys.
</div>

So, what do the items in each category's look like? Coffee is described differently than a book in a shop, so each value might have a different type describing these attributes. So, we might consider making a custom type for each. But, to keep it simple, we will only track for each item its name and its price. The name would be typed as a `String` and initially, we will type the price as a `Double`. We choose a `Double` here because we want to track a more precise price for the items and later have the opportunity to round it accordingly.

```json
{
    "name": "A book about the connection of good coffee and music"
    "price": 10
}
```

<div class="alert alert-block alert-info">
    Notice that a <code>Double</code> has fixed precision and may lead to rounding errors in the program, this will be addressed later.
</div>

Besides the core of our program keeping track of the shopping basket, we would also like a minimal interface which uses IO to operate our program.

The list of action available should be presented when running the program, and we would like to have the actions:
```
Welcome to the store. Possible actions are:
- list_commands
- list_groups
- list_items  --group
- buy_item  --group  --item_index  --quantity
- list_basket
- remove_item  --group  --item_index  --quantity
- get_price  --membership_scheme
- exit
```

Here we call the text after the `--` a flag. This indicates that an extra argument/option must be given to execute that action. Each action should mutate the current shopping basket given the action and return again to the list of possible actions.

As mentioned, we wanted a membership program in the store, this is also reflected in the `get_price --membership_scheme` action with its flag. We call these three membership schemes: "basic", "gold", "platinum", and members will get the following rates
- (Basic 10% | Gold 20% | Platinum 30% ) off for "music" store items
- (Basic 20% | Gold 30% | Platinum 40% ) off for "books" store items
- (Basic 30% | Gold 40% | Platinum 50% ) off for "coffee" store items

### Implementing the code

First, we create a simple mock database by filling it with some mock items.

In [ ]:
import Data.Map as Map

data StoreItem = Item 
                  { name :: String
                  , price :: Double
                  } deriving Show

music1 :: StoreItem
music1 = Item { name = "ABBA greatest hits"
              , price = 80/9
              }

music2 :: StoreItem
music2 = Item { name = "Beatles greatest hits"
              , price = 100/9
              }

music3 :: StoreItem
music3 = Item { name = "Coldplay greatest hits"
              , price = 120/9
              }

music :: Map.Map Int StoreItem
music = Map.fromList [(1,music1),(2,music2),(3,music3)]

book1 :: StoreItem
book1 = Item { name = "English vocabulary"
             , price = 230/9
             }

book2 :: StoreItem
book2 = Item { name = "German vocabulary"
             , price = 240/9
             }

book3 :: StoreItem
book3 = Item { name = "Japanese vocabulary"
             , price = 250/9
             }

books :: Map.Map Int StoreItem
books = Map.fromList [(1,book1),(2,book2),(3,book3)]

coffee1 :: StoreItem
coffee1 = Item { name = "Cappuccino"
               , price = 13/9
               }

coffee2 :: StoreItem
coffee2 = Item { name = "Latte macchiato"
               , price = 11/9
               }

coffee3 :: StoreItem
coffee3 = Item { name = "Espresso "
               , price = 12/9
               }

coffee :: Map.Map Int StoreItem
coffee = Map.fromList [(1,coffee1),(2,coffee2),(3,coffee3)]

storeDatabase :: Map.Map String (Map.Map Int StoreItem)
storeDatabase = Map.fromList [("music",music),("books",books),("coffee",coffee)]

Now that we have our database, let's write the code that interacts with this database. The entry point of our program is the `main` function that will display the welcome message to the user and print the available commands. This function will also introduce an empty list `init_state` that represented the initial empty shopping basket.

The workflow of what follows is the recursively handling of actions that act and update the last state of the shopping basket. We name the function that we will recursively call, the `startShoping` function. This function will also parse user input and act on this input. 

In the `startShoping` function, the following function will represent some of the different actions

- `listCommands` (displays possible commands)

- `listGroups` (displays shopping groups)

- `listBasket` (displays the basket items)

- `wrongCommand` (displays an info message to the user)

For now, we do not implement the other actions and just return the initial basket for these functions to showcase the framework. After this, we will implement the rest of the functions.

In [ ]:
main :: IO ()
main = do
    putStrLn "Welcome to the store."
    printCommands

    let init_state = []
    startShoping init_state

printCommands :: IO ()
printCommands = do
    putStrLn "Possible actions are:\n \
            \ list_commands \n \
            \ list_groups \n \
            \ list_items  --group \n \
            \ buy_item  --group  --item_index  --quantity \n \
            \ list_basket \n \
            \ remove_item  --group  --item_index  --quantity \n \
            \ get_price  --membership_scheme \n \
            \ exit \n\n \
            \ Example command: list_items music"

type CommandOptions = [String]
type ShoppingGroup = String
type ItemIndex = Int
type ItemQuantity = Int
type ShoppingBasket = [(ShoppingGroup,ItemIndex,ItemQuantity)]

startShoping :: ShoppingBasket -> IO ()
startShoping basket = do
    putStrLn "\nPlease select what you want to do: "
    fullCommand <- getLine
    let (command:options) = words fullCommand

    if command == "exit"
    then putStrLn "Exiting shopping."
    else do
        let function = case command of
                "list_commands" -> listCommands
                "list_groups" -> listGroups
                "list_items" -> listItems
                "buy_item" -> buyItem
                "list_basket" -> listBasket
                "remove_item" -> removeItem
                "get_price" -> getPrice
                _ -> wrongCommand

        updatedBasket <- function basket options
        startShoping updatedBasket

listCommands :: ShoppingBasket -> CommandOptions -> IO ShoppingBasket
listCommands basket options = do
    printCommands
    return basket

listGroups :: ShoppingBasket -> CommandOptions -> IO ShoppingBasket
listGroups basket options = do
    putStrLn "Shopping groups are: music, books, coffee."
    return basket

listItems :: ShoppingBasket -> CommandOptions -> IO ShoppingBasket
listItems basket options = do
    return basket

buyItem :: ShoppingBasket -> CommandOptions -> IO ShoppingBasket
buyItem basket options = do
    return basket

listBasket :: ShoppingBasket -> CommandOptions -> IO ShoppingBasket
listBasket basket options = do
    print basket
    return basket

removeItem :: ShoppingBasket -> CommandOptions -> IO ShoppingBasket
removeItem basket options = do
    return basket

getPrice :: ShoppingBasket -> CommandOptions -> IO ShoppingBasket
getPrice basket options = do
    return basket

wrongCommand :: ShoppingBasket -> CommandOptions -> IO ShoppingBasket
wrongCommand basket options = do
    putStrLn "The command you entered is not correct."
    return basket

First, we implement the function `listItems` that lists the basket items and their indexes. We let the user know if the command option he typed in is incorrect, or if he typed in as many or no options.

We use the `notElem` function that works the same as the `elem` function, just that is says True if the element is not contained in the list.

In [ ]:
listItems :: ShoppingBasket -> CommandOptions -> IO ShoppingBasket
listItems basket options = do
    if length options /= 1
    then do
        putStrLn "To much or to less options for this command."
        return basket
    else do
        let group = head options
        if group `notElem` ["music","books","coffee"]
        then do
            putStrLn "This group does not exist."
            return basket
        else do
            let groupData = case group of
                    "music" -> music
                    "books" -> books
                    "coffee" -> coffee
                    _ -> Map.fromList []
            print (Map.toList groupData)
            return basket

Next, we implement the function `buyItem` that lets us add an item to the basket.

We use the function `readMaybe` from the **Text.Read** module and some helper functions to work with the `Maybe` type.

The `readMaybe` function takes a string and returns a maybe value. In the case the string can be read as a number, the function returns a Just value of the number, else it returns Nothing.
```haskell
readMaybe :: Read a => String -> Maybe a
```

We make the following checks on the command parameters and notify the user if any fails:
- is the count of the supply command parameters correct

- are the command parameters legal values

We also check if the item we want to add already exists in the basket and update the quantity accordingly.

In [ ]:
import Text.Read
import Data.Maybe

buyItem :: ShoppingBasket -> CommandOptions -> IO ShoppingBasket
buyItem basket options = do
    if length options /= 3
    then do
        putStrLn "To much or to less options for this command."
        return basket
    else do
        let (group:itemIndex:quantity:_) = options
            groupCheck = group `elem` ["music","books","coffee"]
            itemIndexCheck = let maybeInt = readMaybe itemIndex :: Maybe Int
                                 integerCheck = isJust maybeInt
                             in
                                 if integerCheck then (fromJust maybeInt) `elem` [1,2,3] else False
            quantityCheck = let maybeInt = readMaybe quantity :: Maybe Int
                                integerCheck = isJust maybeInt
                            in
                                if integerCheck then (fromJust maybeInt > 0) else False
        if groupCheck && groupCheck && quantityCheck
        then do
            let itemIndexNum = read itemIndex :: Int
                quantityNum = read quantity :: Int
                checkItem = (\(x,y,_) -> x == group && y == itemIndexNum)
                existingItem = Prelude.filter checkItem basket
            if length existingItem == 1
            then do
                let (_,_,existingQuantity) = head existingItem
                    newQuantity = existingQuantity + quantityNum
                    updatedBasket = Prelude.filter (not . checkItem) basket ++ [(group, itemIndexNum, newQuantity)]
                return updatedBasket
            else do
                let updatedBasket = basket ++ [(group, itemIndexNum, quantityNum)]
                return updatedBasket
        else do
            putStrLn "One or more of the options have not a legal value."
            return basket

Then we implement the `removeItem` function that lowers the amount of an item in the basket or deletes the item entirely.

We use the same functions and do the same checks as in the previous function `buyItem`.

In [ ]:
removeItem :: ShoppingBasket -> CommandOptions -> IO ShoppingBasket
removeItem basket options = do
    if length options /= 3
    then do
        putStrLn "To much or to less options for this command."
        return basket
    else do
        let (group:itemIndex:quantity:_) = options
            groupCheck = group `elem` ["music","books","coffee"]
            itemIndexCheck = let maybeInt = readMaybe itemIndex :: Maybe Int
                                 integerCheck = isJust maybeInt
                             in
                                 if integerCheck then (fromJust maybeInt) `elem` [1,2,3] else False
            quantityCheck = let maybeInt = readMaybe quantity :: Maybe Int
                                integerCheck = isJust maybeInt
                            in
                                if integerCheck then (fromJust maybeInt > 0) else False
        if groupCheck && groupCheck && quantityCheck
        then do
            let itemIndexNum = read itemIndex :: Int
                quantityNum = read quantity :: Int
                checkItem = (\(x,y,_) -> x == group && y == itemIndexNum)
                existingItem = Prelude.filter checkItem basket
            if length existingItem == 1
            then do
                let (_,_,existingQuantity) = head existingItem
                    newQuantity = existingQuantity - quantityNum
                    updatedBasket = let filteredBasket = Prelude.filter (not . checkItem) basket
                                    in
                                        if newQuantity > 0
                                        then filteredBasket ++ [(group, itemIndexNum, newQuantity)]
                                        else filteredBasket
                return updatedBasket
            else do
                putStrLn "Item does not exist in basket."
                return basket
        else do
            putStrLn "One or more of the options has not a legal value."
            return basket

<div class="alert alert-block alert-info">
    <b>CHALLENGE:</b> Also if you look at the code above and compare it to the previous function you see that 4 out of 6 do-blocks are the same. Try to rewrite the above functions in such a way that you extract the common part as much as possible and call it from those functions.
</div>

The last function we implement is `getPrice`, which calculates from our basket the price we have to pay and displays it to the user.

First, we make some checks if the options for the command are correct and let the user know if they are not.

If everything is ok, we call the `printPrice` function which calculates the price of the basket items, taking their quantity and membership discount in account.

In [ ]:
getPrice :: ShoppingBasket -> CommandOptions -> IO ShoppingBasket
getPrice basket options = do
    if length options > 1
    then do
        putStrLn "To much options for this command."
        return basket
    else do
        if length options == 0
        then printPrice basket options
        else if head options `notElem` ["basic", "gold", "platinum"]
             then do
                 putStrLn "Incorrect option for this command."
                 return basket
             else do
                 printPrice basket options

printPrice :: ShoppingBasket -> CommandOptions -> IO ShoppingBasket
printPrice basket options = do
    let membership = if Prelude.null options
                     then ""
                     else head options
        discountCoef x y = if Prelude.null options
                           then 1
                           else discount x y
        calculatePrice (group,itemIndex,quantity) = fromIntegral quantity * 
                                                        discountCoef membership group *
                                                            (price $ fromJust 
                                                                (Map.lookup itemIndex $ fromJust 
                                                                    (Map.lookup group storeDatabase)))
        basketPrice = sum $ Prelude.map calculatePrice basket
    putStrLn $ "Price for your basket is: " ++ show basketPrice
    return basket
  where discount membership grp = let membershipFactor = case membership of
                                                             "basic" -> 0
                                                             "gold" -> 1
                                                             "platinum" -> 2
                                  in case grp of
                                         "music" -> 0.9 - 0.1*membershipFactor
                                         "books" -> 0.8 - 0.1*membershipFactor
                                         "coffee" -> 0.7 - 0.1*membershipFactor

We get some rounding errors because the `Double` type can handle only 16 digits of precision, when the lead decimal number is not a 0.

Let us try to solve this problem by searching for a more accurate type than `Double` and comparing the results.

### Minimizing rounding errors

If we google for *Haskell number types* we find that the built-in number types are `Num, Real, Integral, Integer, Int, Ratio, Rational, Double, Float`.

Because we need larger decimal precision than the `Double` type has to offer, we choose the `Rational` type to re-implement our code.

We test this type with the code below, where we can see how this type can help us to preserve precision.

In [ ]:
-- without loosing precision
fromRational ((7/25 :: Rational)*25) :: Double

-- loosing some precision
(fromRational (7/25 :: Rational)) * 25 :: Double

We need only to change the type of the price field in the `StoreItem` data type from `Double` to `Rational`. 

In [ ]:
data StoreItem = Item 
                  { name :: String
                  , price :: Rational
                  } deriving Show

music1 :: StoreItem
music1 = Item { name = "ABBA greatest hits"
              , price = 80/9
              }

music2 :: StoreItem
music2 = Item { name = "Beatles greatest hits"
              , price = 100/9
              }

music3 :: StoreItem
music3 = Item { name = "Coldplay greatest hits"
              , price = 120/9
              }

music :: Map.Map Int StoreItem
music = Map.fromList [(1,music1),(2,music2),(3,music3)]

book1 :: StoreItem
book1 = Item { name = "English vocabulary"
             , price = 230/9
             }

book2 :: StoreItem
book2 = Item { name = "German vocabulary"
             , price = 240/9
             }

book3 :: StoreItem
book3 = Item { name = "Japanese vocabulary"
             , price = 250/9
             }

books :: Map.Map Int StoreItem
books = Map.fromList [(1,book1),(2,book2),(3,book3)]

coffee1 :: StoreItem
coffee1 = Item { name = "Cappuccino"
               , price = 13/9
               }

coffee2 :: StoreItem
coffee2 = Item { name = "Latte macchiato"
               , price = 11/9
               }

coffee3 :: StoreItem
coffee3 = Item { name = "Espresso "
               , price = 12/9
               }

coffee :: Map.Map Int StoreItem
coffee = Map.fromList [(1,coffee1),(2,coffee2),(3,coffee3)]

storeDatabase :: Map.Map String (Map.Map Int StoreItem)
storeDatabase = Map.fromList [("music",music),("books",books),("coffee",coffee)]

After that, we have to update the `printPrice` function such that it works with `Rational` type numbers and prints the result in `Double`.

In [ ]:
printPrice :: ShoppingBasket -> CommandOptions -> IO ShoppingBasket
printPrice basket options = do
    let membership = if Prelude.null options
                     then ""
                     else head options
        discountCoef x y = if Prelude.null options
                           then 1
                           else discount x y
        calculatePrice (group,itemIndex,quantity) = fromIntegral quantity * 
                                                        discountCoef membership group *
                                                            (price $ fromJust 
                                                                (Map.lookup itemIndex $ fromJust 
                                                                    (Map.lookup group storeDatabase)))
        basketPrice = fromRational (sum $ Prelude.map calculatePrice basket) :: Double
    putStrLn $ "Price for your basket is: " ++ show basketPrice
    return basket
  where discount membership grp = let membershipFactor = case membership of
                                                             "basic" -> 0
                                                             "gold" -> 1
                                                             "platinum" -> 2
                                  in case grp of
                                         "music" -> (9 - 1*membershipFactor)/10 :: Rational
                                         "books" -> (8 - 1*membershipFactor)/10 :: Rational
                                         "coffee" -> (7 - 1*membershipFactor)/10 :: Rational

We expect that we will lose some precision in the final rounding, but not in the steps when we calculate the final price by using the `Rational` type.

We define the following case of shopping:
- buy_item music 1 4
- buy_item books 2 5
- buy_item coffee 3 6

The final basket prices for the `Double` and `Rational` types for all memberships are:
- get_price basic
  <br>Price for your basket is: 144.26666666666668   (Double)
  <br>Price for your basket is: 144.26666666666668   (Rational)

- get_price gold
  <br>Price for your basket is: 126.57777777777778   (Double)
  <br>Price for your basket is: 126.57777777777778   (Rational)

- get_price platinum
  <br>Price for your basket is: 108.8888888888889    (Double)
  <br>Price for your basket is: 108.88888888888889   (Rational)

The only case where we profit some precision is for the platinum membership. The other two cases give us the same result if in the end convert the `Rational` result to `Double`.

We are still not happy and google for *Haskell rounding error*. We see there is another type called `Decimal`. 

On Hoogle, if we choose the **Data.Decimal** module, we see that the smallest number for a Decimal type is 10^-255.

To make use of this module, you need to install the **Decimal** package using cabal.

The `Decimal` type is, in general, more precise as the `Double` type but also here sometimes rounding errors appear on 255 decimal places, as in the code below.

In [ ]:
import Data.Decimal 

a = (25/7) :: Decimal
print a
b = a * 7
print b

Let's re-write our `StoreItem` data type so that we use the `Decimal` type for the store item prices.

In [ ]:
import Data.Decimal

data StoreItem = Item 
                  { name :: String
                  , price :: Decimal
                  } deriving Show

Here we do not have to update the `printPrice` function. We can use the same version that we used in the begining for the `Double` type.

We again pick the same shopping case:
- buy_item music 1 4
- buy_item books 2 5
- buy_item coffee 3 6

The results for all three types are as follows:
- get_price basic
  <br>Price for your basket is: 144.26666666666668   (Double)
  <br>Price for your basket is: 144.26666666666668   (Rational)
  <br>Price for your basket is: 144.26666666666666...<254 times 6>...67 (Decimal)

- get_price gold
  <br>Price for your basket is: 126.57777777777778   (Double)
  <br>Price for your basket is: 126.57777777777778   (Rational)
  <br>Price for your basket is: 126.57777777777777...<254 times 7>...78 (Decimal)

- get_price platinum
  <br>Price for your basket is: 108.8888888888889    (Double)
  <br>Price for your basket is: 108.88888888888889   (Rational)
  <br>Price for your basket is: 108.8888888888888...<254 times 8>...89  (Decimal)

We see that the `Decimal` type gives us the most accurate precision for our case. 

In this example, the rounding errors are not that big for a store that has many customers the errors sum up and in the end can make a significant difference.

## Recap

In this lesson, we have discussed:

- how you can help yourself with the `:i` and `:t` commands

- how to help yourself with **Hoogle** and **Hackage**

- we mentioned where to find additional learning resources

- we did a walkthrough of looking at existing code that uses the **Data.Map.Strict** module

- we did a walkthrough of a project where we learned how to minimize rounding errors